In [ ]:
import os
import sys

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../../Python-Table/task2'))

import numpy as np
import matplotlib.pyplot as plt
import idx2numpy
import pandas as pd
from lib.Net import *
from lib.Func import *

In [ ]:
# (batch_size, 1, 28, 28) -> (batch_size, 2, 24, 24) -> (batch_size, 2*24*24) -> (batch_size, 64) -> (batch_size, 10)
architecture = [
    {"module": Conv2d, "params": {"in_channels": 1, "out_channels": 2, "kernel_size": 5, "activation": "gelu"}},
    {"module": Flatten},
    {"module": Mlp, "params": {"input_dim": 1152, "output_dim": 64, "activation": "gelu"}},
    {"module": Dropout, "params": {"p": 0.3}},
    {"module": Mlp, "params": {"input_dim": 64, "output_dim": 10, "activation": "softmax"}},
]

In [ ]:
net = Net(architecture)

In [ ]:
datapath = "../data/MNIST/train-images.idx3-ubyte"
labelpath = "../data/MNIST/train-labels.idx1-ubyte"
modelpath = "../model/task2/MNIST.json"

In [ ]:
data = idx2numpy.convert_from_file(datapath)    # (60000, 28, 28)
data = np.expand_dims(data, axis=1)             # 添加通道维度 -> (60000, 1, 28, 28)

label = idx2numpy.convert_from_file(labelpath)
one_hot_labels = one_hot(label, 10)

In [ ]:
epochs = 1
pbar =tqdm(range(epochs))

for i in pbar:
    # data, one_hot_labels = shuffle(data, one_hot_labels)
    net.train(data, one_hot_labels, batch_size=512, lr=0.1, lossfunc="cross_entropy")

    y_hat = net.predict(data)
    loss = cross_entropy_loss(one_hot_labels, y_hat)
    y_hat = np.argmax(y_hat, axis=1)    # (60000, 10) -> (60000,)
    accuracy = np.mean(y_hat == one_hot_labels.argmax(axis=1))
    pbar.set_postfix({"loss": loss, "accuracy": f"{accuracy*100:.2f}%"})

Batch 78/117, Loss: 4.1923
Batch 79/117, Loss: 4.1295
Batch 80/117, Loss: 4.2193
Batch 81/117, Loss: 4.1385
Batch 82/117, Loss: 4.1385
Batch 83/117, Loss: 4.2552
Batch 84/117, Loss: 4.1116
Batch 85/117, Loss: 4.2372
Batch 86/117, Loss: 4.2103
Batch 87/117, Loss: 4.2462
Batch 88/117, Loss: 4.0846
Batch 89/117, Loss: 4.0308
Batch 90/117, Loss: 4.1744
Batch 91/117, Loss: 4.2103
Batch 92/117, Loss: 4.1564
Batch 93/117, Loss: 4.1834
Batch 94/117, Loss: 4.1923
Batch 95/117, Loss: 4.2911
Batch 96/117, Loss: 4.2013
Batch 97/117, Loss: 4.1744
Batch 98/117, Loss: 4.1116
Batch 99/117, Loss: 4.0757
Batch 100/117, Loss: 4.2193
Batch 101/117, Loss: 4.2552
Batch 102/117, Loss: 4.1654
Batch 103/117, Loss: 4.2282
Batch 104/117, Loss: 4.2103
Batch 105/117, Loss: 4.1295
Batch 106/117, Loss: 4.1475
Batch 107/117, Loss: 4.1295
Batch 108/117, Loss: 4.2103
Batch 109/117, Loss: 4.0936
Batch 110/117, Loss: 4.0757
Batch 111/117, Loss: 4.1834
Batch 112/117, Loss: 4.0846
Batch 113/117, Loss: 4.2462
Batch 114/117,

In [ ]:
#net.save_params(modelpath_2)